# ASG Microfund

This notebook includes a detailed explination of the ASG Microfund and the inner working of the system. With a step by step breakdown of the process we can see that this fund can be modularly added and edited

## Step 0: Dependancies and Libraries

Full list gotten from requirments.txt

## Step 1: Main.py

Here is where youb run the script from. The variables are established here and carried forward throughout the system

In [ ]:
start_date = '2020-01-01'
end_date = '2025-01-01'
user_risk_tol = 'medium'
user_time_hor = 'medium'
mean_tickers = ['CL=F']
tickers = pd.read_html("https://en.wikipedia.org/wiki/List_of_S%26P_500_companies")[0]['Symbol'].tolist()
factor_tickers = [t.replace('.', '-') for t in tickers]
commissions = 0.001
cash = 1000000


generate_report(start_date, end_date, user_risk_tol, user_time_hor, mean_tickers, factor_tickers, commissions, cash)


#----------- IN PROGRESS ------------
#generate_pdf(start_date, end_date, user_risk_tol, user_time_hor) 

#----------- TO RUN STREAMLIT DASHBOARD -----------

# streamlit run dashboard/streamlit_app.py

## Step 2: Generating the Report

This is the main function in this project, here we can generate a html report using the jinja template. The variables in the template are filled in using the rest of the projects code. When selected to run, generate_report calls the rest of the functions and classes built here

In [ ]:

def _render_html(start_date, end_date,risk, time, mean_tickers, factor_tickers, commissions, cash):

    template_env = jinja2.Environment(
        loader=jinja2.FileSystemLoader(os.path.join(os.path.dirname(__file__), "templates")),
        autoescape=jinja2.select_autoescape(['html', 'xml'])
    )

    try:
        template = template_env.get_template("monthly_report_template.html")
    except jinja2.TemplateNotFound:
        raise FileNotFoundError("Template 'monthly_report_template.html' not found in ./reporting/templates")

    output_dir = os.path.join("reporting", "reports")
    os.makedirs(output_dir, exist_ok=True)

    output_filename = f"ASG Microfund - {start_date} to {end_date}.html"
    output_path = os.path.join(output_dir, output_filename)

    port = Portfolio('2020-01-01', '2025-01-01', commissions, cash, mean_tickers, factor_tickers, risk,  time)
    mean_reversion_summary, momentum_summary, factor_summary, final_metrics, benchmark_summary, returns_df= port.backtest_portfolio()
    factor_plot_dir, mom_plot_dir, mean_plot_dir, equity_curve_path, daily_return_path = port.plot_stratgies()

    benchmark_metrics_summary = {'Return [%]': benchmark_summary['Return [%]'], 'CAGR [%]': benchmark_summary['CAGR [%]'], 'Sharpe Ratio': benchmark_summary['Sharpe Ratio'], 'Max. Drawdown': benchmark_summary['Max. Drawdown [%]']}


    factor_plot_path_web = os.path.relpath(factor_plot_dir, start=os.path.dirname(output_path)).replace('\\', '/')
    momentum_plot_path_web = os.path.relpath(mom_plot_dir, start=os.path.dirname(output_path)).replace('\\', '/')
    mean_plot_web = os.path.relpath(mean_plot_dir, start=os.path.dirname(output_path)).replace("\\", "/")
    equity_curve_plot_web = os.path.relpath(equity_curve_path, start=os.path.dirname(output_path)).replace("\\", "/")
    daily_returns_plot_web = os.path.relpath(daily_return_path, start=os.path.dirname(output_path)).replace("\\", "/")


    rendered_html = template.render(
        mean_reversion_summary=mean_reversion_summary,
        momentum_summary=momentum_summary,
        factor_summary = factor_summary,
        final_metrics = final_metrics,
        benchmark_summary = benchmark_metrics_summary,
        mean_tickers = mean_tickers,
        factor_tickers = factor_tickers,
        start_date=start_date,
        end_date=end_date,
        factor_plot_dir=factor_plot_path_web,
        mom_plot_dir=momentum_plot_path_web,
        mean_plot_dir=mean_plot_web,
        equity_curve_dir=equity_curve_plot_web,
        daily_return_dir=daily_returns_plot_web
    )
    return rendered_html, output_path

def generate_report(start_date, end_date, risk, time, mean_tickers, factor_tickers, commissions, cash):
    """
    Generates an HTML performance report for a strategy over a given time range.
    
    Parameters:
        strategy (object): The strategy instance.
        start_date (str): Start date in 'YYYY-MM-DD' format.
        end_date (str): End date in 'YYYY-MM-DD' format.
    """

    # Set up the Jinja2 environment
    
    rendered_html, output_path= _render_html(start_date, end_date, risk, time, mean_tickers, factor_tickers, commissions, cash)

    with open(output_path, "w", encoding="utf-8") as f:
        f.write(rendered_html)

    
    print(f"✅ Report generated and saved to {output_path}")

def generate_pdf(start_date, end_date, risk, time):
    """
    Generates a PDF performance report for a strategy over a given time range.
    
    Parameters:
        start_date (str): Start date in 'YYYY-MM-DD' format.
        end_date (str): End date in 'YYYY-MM-DD' format.
        risk (float): Risk parameter for the portfolio.
        time (str): Time horizon for the strategies.
    """
    # First render the HTML content
    rendered_html, html_output_path = _render_html(start_date, end_date, risk, time)
    
    # Set up output directory for PDF
    output_dir = os.path.join("docs", "external")
    os.makedirs(output_dir, exist_ok=True)
    
    # Create PDF output path
    pdf_output_filename = f"ASG Microfund - {start_date} to {end_date}.pdf"
    pdf_output_path = os.path.join(output_dir, pdf_output_filename)
    
    # Options for PDF generation
    options = {
        'page-size': 'A4',
        'margin-top': '0.75in',
        'margin-right': '0.75in',
        'margin-bottom': '0.75in',
        'margin-left': '0.75in',
        'encoding': 'UTF-8',
        'quiet': '',
        'no-outline': None,
        'enable-local-file-access': None  # Required to access local images
    }
    config =imgkit.config(wkhtmltoimage=r"C:\Program Files\wkhtmltopdf\bin\wkhtmltoimage.exe")

    try:
        # Generate PDF from the HTML string
        pdfkit.from_string(rendered_html, pdf_output_path, options=options, configuration=config)
        print(f"✅ PDF report generated and saved to {pdf_output_path}")
        return pdf_output_path
    except Exception as e:
        print(f"❌ Failed to generate PDF report: {str(e)}")
        raise

## Step 3: Portfolio

Here is where the generate report initalizes eveyrthing that will be ran later on. This class has access to the trading stratgies and the libraries. The bulk of the backtesting is also done here using the portfolio.run() method

In [ ]:


class Portfolio:

    def __init__(self, start, end, commissions, cash, mean_tickers, factor_tickers, user_tolerance: str='low', user_time: str='medium'):
        self.risk_manager = RiskManagement(user_tolerance, user_time)

        self.mean_alloc, self.momentum_alloc, self.factor_alloc = self._get_strategy_allocations()

        self.mean_tickers = mean_tickers
        self.factor_tickers = factor_tickers

        self.start = start
        self.end = end
        self.commissions = commissions
        self.cash = cash

        self.risk_manager = RiskManagement(user_tolerance, user_time)
        self.benchmark = benchmark(start=start, end=end)
        self.data_loader = data_loader()

        self.factor_strategy = factor_investing_strategy(start, end, commissions, self.data_loader, self.factor_tickers)
        self.momentum_strategy = momentum_strategy(start, end, self.momentum_alloc, commissions)
        self.mean_strategy = mean_reversion_strategy


    def _get_strategy_allocations(self):
        risk_profile = self.risk_manager.get_risk_profile()
        mean_alloc = risk_profile['allocations_advanced']['mean_reversion']
        momentum_alloc = risk_profile['allocations_advanced']['momentum']
        factor_alloc = risk_profile['allocations_advanced']['factor_investing']

        return mean_alloc, momentum_alloc, factor_alloc
    
    def backtest_mean(self):
        engine = GenericBacktestEngine(
                    strategy_cls=self.mean_strategy,
                    cash=self.mean_alloc*self.cash,
                    commission=self.commissions
                )    
        data = self.data_loader.get_multiple_data(self.mean_tickers, self.start, self.end)
        mean_reversion_results, pf_ret_dict = engine.batch_backtest(data)
        first_df = list(data.values())[0]
        self.mean_plot_path = engine.plot(first_df)
        
        return mean_reversion_results, pf_ret_dict
    
    def backtest_momentum(self):
        momentum_results, pf_ret = self.momentum_strategy.run()
        return momentum_results, pf_ret
    
    def backtest_factor(self):
        pf_cum, pf_ret = self.factor_strategy.backtest()
        metrics = self.factor_strategy.get_metrics()
        return metrics, pf_ret
    
    def _get_portfolio_results(self, mean_reversion_results, momentum_results, factor_results):

        final_metrics = {}
        metrics = ['Return [%]', 'CAGR [%]', 'Sharpe Ratio', 'Max. Drawdown [%]']

        metric_values = {metric: [] for metric in metrics}
        traded_stocks = 0

        sorted_results = sorted(mean_reversion_results.items(), key=lambda x: x[1]['Return [%]'], reverse=True)
        

        for ticker, stat in sorted_results:
            if stat['Return [%]'] != 0:
                traded_stocks += 1
                for metric in metrics:
                    val = stat.get(metric, np.nan)
                    metric_values[metric].append(val)

        summary_stats = {}

        for metric, values in metric_values.items():
            clean_vals = [v for v in values if not (isinstance(v, float) and np.isnan(v))]
            summary_stats[metric] = {
                'avg': np.mean(clean_vals) if clean_vals else np.nan,
                'median': np.median(clean_vals) if clean_vals else np.nan
            }

        mean_reversion_summary =  summary_stats

        momentum_summary = {metric: momentum_results[metric] for metric in metrics if metric in momentum_results}

        factor_summary = factor_results

        for metric in metrics:
            # Extract values from each summary
            mean_val = mean_reversion_summary.get(metric, {}).get('avg', np.nan)
            momentum_val = momentum_summary.get(metric, np.nan)
            factor_val = factor_summary.get(metric, np.nan)

            # Weighted average of available metrics
            weighted_values = [
                mean_val * self.mean_alloc if not np.isnan(mean_val) else 0,
                momentum_val * self.momentum_alloc if not np.isnan(momentum_val) else 0,
                factor_val * self.factor_alloc if not np.isnan(factor_val) else 0
            ]
            total_weight = sum([
                self.mean_alloc if not np.isnan(mean_val) else 0,
                self.momentum_alloc if not np.isnan(momentum_val) else 0,
                self.factor_alloc if not np.isnan(factor_val) else 0
            ])
            final_metrics[metric] = sum(weighted_values) / total_weight if total_weight > 0 else np.nan

        return mean_reversion_summary, momentum_summary, factor_summary, final_metrics
    

    def returns_df(self, mean_reversion, momentum, factor_investing):
        """
        Combine daily returns from 3 strategies into one aligned DataFrame.
        
        Parameters:
            mean_reversion (dict): Daily returns (only use first).
            momentum (pd.Series): Monthly returns.
            factor_investing (pd.Series): Daily returns.
            
        Returns:
            pd.DataFrame: Combined returns with columns:
                        ['Mean Reversion', 'Factor Investing', 'Momentum']
        """
        
        if not mean_reversion:
            raise ValueError("mean_reversion dictionary is empty")
        
        # Get first mean reversion series
        first_key = next(iter(mean_reversion))
        mean_reversion = pd.Series(mean_reversion[first_key])

        monthly_dates = pd.date_range(start=self.start, periods=len(momentum), freq='ME')
        momentum.index = monthly_dates        
        # Create common index
        start_date = min(
            #momentum.index.min(), 
            mean_reversion.index.min(), 
            factor_investing.index.min()
        )
        end_date = max(
            #momentum.index.max(), 
            mean_reversion.index.max(), 
            factor_investing.index.max()
        )
        full_index = pd.date_range(start=start_date, end=end_date, freq='D')
        
        combined = pd.DataFrame(index=full_index)
        
        combined['Mean Reversion'] = mean_reversion
        combined['Factor Investing'] = factor_investing
        
        
        momentum_daily_value = momentum.reindex(full_index, method='ffill')
        combined['Momentum'] = momentum_daily_value.pct_change().fillna(0)

        combined.dropna(how='any', inplace=True)
        self.combined_results = combined
        return combined
    
    def _plot_daily_returns(self, save_dir="reporting/charts"):
        df_returns = self.combined_results
        allocs = {
        'Mean Reversion': self.cash * self.mean_alloc,
        'Momentum': self.cash * self.momentum_alloc,
        'Factor Investing': self.cash * self.factor_alloc
        }  

        equity_curves = {}
        for strat in df_returns.columns:
            cumulative_return = (1 + df_returns[strat]).cumprod()
            equity_curves[strat] = cumulative_return * allocs[strat]

        df_equity = pd.DataFrame(equity_curves)

        plt.figure(figsize=(12, 6))
        for strat in df_equity.columns:
            plt.plot(df_equity.index, df_equity[strat], label=strat, linewidth=2)
        plt.title("Equity Curve per Strategy")
        plt.xlabel("Date")
        plt.ylabel("Portfolio Value ($)")
        plt.grid(True)
        plt.legend()

        ax = plt.gca()
        ax.get_yaxis().set_major_formatter(
            mtick.FuncFormatter(lambda x, p: f'${int(x):,}')
        )

        equity_path = os.path.join("reporting", "charts", 'equity_curve.png')
        os.makedirs(os.path.dirname(equity_path), exist_ok=True)
        plt.savefig(equity_path)
        plt.close()
                    
        # --- Plot Daily Returns ---
        plt.figure(figsize=(12, 6))
        for strat in df_returns.columns:
            plt.plot(df_returns.index, df_returns[strat], label=strat, linewidth=1)
        plt.title("Daily Returns per Strategy")
        plt.xlabel("Date")
        plt.ylabel("Daily Return")
        plt.grid(True)
        plt.legend()
        if df_returns.abs().max().max() > 0.2:
            plt.ylim(-0.20, 0.20)

        plt.tight_layout()
        returns_path = os.path.join("reporting", "charts", "daily_returns.png")
        os.makedirs(os.path.dirname(returns_path), exist_ok=True)
        plt.savefig(returns_path)
        plt.close()

        return equity_path, returns_path


    def backtest_portfolio(self):
        mean_reversion_results, mean_returns = self.backtest_mean()
        momentum_results, momentum_returns = self.backtest_momentum()
        factor_results, factor_returns = self.backtest_factor()

        mean_reversion_summary, momentum_summary, factor_summary, final_metrics = self._get_portfolio_results(mean_reversion_results, momentum_results, factor_results)
        benchmark_results = self.benchmark.get_metrics()
        returns_df = self.returns_df(mean_returns, momentum_returns, factor_returns)
        return mean_reversion_summary, momentum_summary, factor_summary, final_metrics, benchmark_results, returns_df


    def plot_stratgies(self):
        factor_plot_path = self.factor_strategy.plot_performance()
        momentum_plot_path = self.momentum_strategy.plot_preformance()
        mean_plot_path = self.mean_plot_path
        equity_curves_path, returns_curve_path = self._plot_daily_returns()
        return factor_plot_path, momentum_plot_path, mean_plot_path, equity_curves_path, returns_curve_path

    






## Step 4: Risk Management

The time horizon and risk tolerance variables entered all the way from the beginning of the main.py file are carried here and entered to retrieve a risk composite score which then distriubte the funds, also allocated in the main.py file to each of the 3 different stratgies.

In [ ]:
class RiskManagement:
    """A class to manage risk allocations and strategy distribution based on user's profile."""
    
    # Class-level constants for better maintainability
    TOLERANCE_MAPPING = {
        'low': 0.2,
        'medium': 0.5,
        'high': 0.8
    }
    
    TIME_MAPPING = {
        'long': 0.2,
        'medium': 0.5,
        'short': 0.8
    }
    
    MAX_RISK_SCORE = TOLERANCE_MAPPING['high'] * TIME_MAPPING['short']

    def __init__(self, user_tolerance: str, user_time: str):
        """Initialize with user's risk tolerance and time horizon.
        
        Args:
            user_tolerance: Risk tolerance ('low', 'medium', or 'high')
            user_time: Time horizon ('long', 'medium', or 'short')
        """
        self.user_tolerance = user_tolerance.lower()
        self.user_time = user_time.lower()
        
        # Validate inputs
        self._validate_inputs()
        self._risk_score = self._get_risk_score()  # Calculate risk score at initialization

    def _validate_inputs(self):
        """Validate that inputs are within expected values."""
        if self.user_tolerance not in self.TOLERANCE_MAPPING:
            raise ValueError(f"Invalid risk tolerance. Expected one of: {list(self.TOLERANCE_MAPPING.keys())}")
            
        if self.user_time not in self.TIME_MAPPING:
            raise ValueError(f"Invalid time horizon. Expected one of: {list(self.TIME_MAPPING.keys())}")

    def _get_risk_score(self) -> float:
        """Calculate and return the risk allocation ratio.
        
        Returns:
            float: Risk allocation ratio between 0 and 1
        """
        tol_num = self.TOLERANCE_MAPPING[self.user_tolerance]
        tim_num = self.TIME_MAPPING[self.user_time]
        
        risk_score = (tol_num * tim_num) / self.MAX_RISK_SCORE
        
        # Ensure the result is between 0 and 1
        return max(0.0, min(1.0, risk_score))

    def get_strategy_allocation(self) -> dict:
        """Calculate allocation percentages for each strategy using curved relationships.
        
        Returns:
            dict: {'mean_reversion': x%, 'momentum': y%, 'factor_investing': z%}
        """
        # Strategy weights based on risk score
        mean_rev = self._risk_score * 0.7  # Short-term strategy scales with risk
        momentum = 0.5 - (0.5 - self._risk_score)**2  # Medium-term peaks in middle
        factor_inv = (1 - self._risk_score) * 0.7  # Long-term scales inversely
        
        # Normalize to 100%
        total = mean_rev + momentum + factor_inv
        return {
            'mean_reversion': mean_rev / total,
            'momentum': momentum / total,
            'factor_investing': factor_inv / total
        }

    def get_risk_profile(self) -> dict:
        """Return comprehensive risk profile including score and allocations.
        
        Returns:
            dict: {
                'risk_score': float,
                'allocations_advanced': dict,
                'allocations_simple': dict
            }
        """
        return {
            'risk_score': self._risk_score,
            'allocations_advanced': self.get_strategy_allocation(),
        }
    
    

## Step 5: Stratgies

This is the bread and butter of the program it does the actual testing and plotting of the trading stratgies. Grabbing the data from data loader after retireving the ticker symbols from the inputs from main.py

### Factor investing

In [ ]:

class factor_investing_screener:

    def __init__(self, data_loader, tickers):
        self.loader = data_loader
        self.tickers = tickers
        self.passed = []

    def _get_features(self):
        for ticker in self.tickers:
            info = self.loader.get_fundamentals(ticker)
            # Check all required keys exist
            required_keys = [
                'priceToBook', 'trailingPE', 'forwardPE', 'enterpriseToEbitda',
                'debtToEquity', 'returnOnEquity', 'returnOnAssets', 'grossMargins',
                'operatingMargins', 'freeCashflow', 'revenueGrowth', 'earningsGrowth',
                'beta', 'marketCap'
            ]

            missing_keys = [k for k in required_keys if k not in info or info[k] is None]
            if missing_keys:
                print(f"[!] Skipping {ticker} — missing keys: {missing_keys}")
                continue


            if all(k in info and info[k] is not None for k in required_keys):
                self.passed.append({
                    'ticker': ticker,
                    'p_b': info['priceToBook'],
                    'pe_ttm': info['trailingPE'],
                    'pe_forward': info['forwardPE'],
                    'ev_ebitda': info['enterpriseToEbitda'],
                    'debt_to_equity': info['debtToEquity'],
                    'roe': info['returnOnEquity'],
                    'roa': info['returnOnAssets'],
                    'gross_margin': info['grossMargins'],
                    'operating_margin': info['operatingMargins'],
                    'fcf': info['freeCashflow'],
                    'revenue_growth': info['revenueGrowth'],
                    'earnings_growth': info['earningsGrowth'],
                    'beta': info['beta'],
                    'market_cap': info['marketCap']
                })
            time.sleep(0.2)  # avoid hitting API rate limits
        return self.passed 
    
    def choose_stocks(self, top_n=10):
        self._get_features()
        df = pd.DataFrame(self.passed)

        if df.empty:
            raise ValueError("No data available to screen.")

        # Invert value metrics: lower is better
        df['p_b_rank'] = df['p_b'].rank(ascending=True)
        df['pe_ttm_rank'] = df['pe_ttm'].rank(ascending=True)
        df['pe_forward_rank'] = df['pe_forward'].rank(ascending=True)
        df['ev_ebitda_rank'] = df['ev_ebitda'].rank(ascending=True)

        # Quality metrics: higher is better
        df['roe_rank'] = df['roe'].rank(ascending=False)
        df['roa_rank'] = df['roa'].rank(ascending=False)
        df['gross_margin_rank'] = df['gross_margin'].rank(ascending=False)
        df['operating_margin_rank'] = df['operating_margin'].rank(ascending=False)

        # Growth metrics: higher is better
        df['revenue_growth_rank'] = df['revenue_growth'].rank(ascending=False)
        df['earnings_growth_rank'] = df['earnings_growth'].rank(ascending=False)

        # Combine all ranks into a composite score (equal weighting for now)
        df['composite_score'] = df[
            [
                'p_b_rank', 'pe_ttm_rank', 'pe_forward_rank', 'ev_ebitda_rank',
                'roe_rank', 'roa_rank', 'gross_margin_rank', 'operating_margin_rank',
                'revenue_growth_rank', 'earnings_growth_rank'
            ]
        ].mean(axis=1)

        df_sorted = df.sort_values(by='composite_score')
        top_stocks = df_sorted.head(top_n).reset_index(drop=True)

        return top_stocks
            

class factor_investing_strategy():

    def __init__(self, start_date, end_date, commission, data_loader, tickers):
        self.start_date = pd.to_datetime(start_date)
        self.end_date = pd.to_datetime(end_date)
        self.commission = commission / 100
        self.screener = factor_investing_screener(data_loader, tickers)
        self.loader = data_loader
        self.portfolio = []
        self.returns = []

    def get_stocks(self, date):
        top_stocks = self.screener.choose_stocks(top_n=3)
        return top_stocks['ticker'].tolist()
    
    def backtest(self):
        tickers = self.get_stocks(self.start_date)
        if not tickers:
            print(f"[!] No stocks selected at start date {self.start_date}.")
            return None
        
        data_dict = self.loader.get_multiple_data(tickers, start=self.start_date, end=self.end_date)
        prices = pd.DataFrame({ticker: df['Close'] for ticker, df in data_dict.items() if 'Close' in df})

        if prices.empty:
            print("[!] No valid price data for the period.")
            return None
        
        prices = prices.dropna(axis=1, how='any')
        if prices.shape[1] == 0:
            print("[!] All stocks had missing data. Exiting.")
            return None
        
        rets = prices.pct_change().dropna()

        pf_ret = rets.mean(axis=1) * (1-self.commission)

        pf_cum = (1+pf_ret).cumprod()

        self.returns = pf_ret
        self.portfolio = pf_cum

        return pf_cum, pf_ret
        


    def plot_performance(self, save_path=None):
        if self.portfolio is None or self.portfolio.empty:
            print("[!] No portfolio performance to plot.")
            return

        plt.figure(figsize=(12, 6))
        plt.plot(self.portfolio, label='Factor Investing Strategy', color='navy')
        plt.title('Factor Investing Strategy Performance')
        plt.xlabel('Date')
        plt.ylabel('Cumulative Return')
        plt.grid(True)
        plt.legend()
        plt.tight_layout()
        if save_path is None:
            save_path = os.path.join('reporting', 'charts', 'factor_investing_plot.png')

        os.makedirs(os.path.dirname(save_path), exist_ok=True)
        plt.savefig(save_path)
        plt.close()

        return save_path

    def get_metrics(self, risk_free_rate=0.02):
        returns = self.returns
        if returns is None or returns.empty:
            return None

        # Correct total return
        total_return = (1 + returns).prod() - 1

        # Annualized return (CAGR)
        ann_ret = (1 + total_return) ** (12 / len(returns.resample('ME').mean())) - 1

        # Annualized volatility
        ann_vol = returns.std() * np.sqrt(252)  # Use 252 if using daily returns

        # Sharpe ratio
        sharpe = (ann_ret - risk_free_rate) / ann_vol if ann_vol != 0 else np.nan

        # Max drawdown
        cum_returns = self.portfolio
        drawdown = cum_returns / cum_returns.cummax() - 1
        max_dd = drawdown.min()

        metrics = {
            'Return [%]': total_return * 100,
            'CAGR [%]': ann_ret * 100,
            'Sharpe Ratio': sharpe,
            'Max. Drawdown [%]': max_dd * 100
        }

        return metrics





### Momentum

In [ ]:

class momentum_strategy():

    LOOKBACK_WINDOWS = [12,6,3]

    def __init__(self, start_date, end_date, equity, commission: float=0.02, index:str="^GSPC", LOOKBACK_WINDOWS: list=[12,6,3]):
        self.start_date = start_date
        self.end_date = end_date
        self.index = index
        self.LOOKBACK_WINDOWS = LOOKBACK_WINDOWS
        self.commission = commission/100
        self.equity = equity

    def _get_data(self):
        dl = data_loader()
        df = dl.get_sp500_data_df(start=self.start_date, end=self.end_date)
        mtl = (df.pct_change() + 1 )[1:].resample("ME").prod()
        return mtl
    
    def _get_rolling_returns(self, mtl, a, b, c):
        f = lambda x: np.prod(x) - 1
        return mtl.rolling(a).apply(f), mtl.rolling(b).apply(f), mtl.rolling(c).apply(f)
    

    def plot_preformance(self, save_path=None):
        strat_pf = self.strat_pf
        if strat_pf is None or strat_pf.empty:
            print("[!] No performance data to plot.")
            return None
        
        plt.figure(figsize=(12, 6))
        plt.plot(strat_pf, label='Momentum Strategy', linewidth=2, color='darkgreen')
        plt.title(f'Momentum Strategy')
        plt.xlabel('Date')
        plt.ylabel('Cumulative Return')
        plt.grid(True)
        plt.legend()
        plt.tight_layout()
        
        if save_path is None:
            save_path = os.path.join("reporting", "charts", 'momentum_strategy_plot.png')
        os.makedirs(os.path.dirname(save_path), exist_ok=True)
        plt.savefig(save_path)
        plt.close()

        return save_path

    def _metrics(self, pf_series, risk_free_rate=0.0):
        df = pf_series.copy()
        df = df.dropna()
        
        start = df.index[0]
        end = df.index[-1]
        duration = end - start

        returns = df.pct_change().dropna()
        cumulative_return = df[-1] / df[0] - 1
        n_years = (df.index[-1] - df.index[0]).days / 365.25

        cagr = (df[-1] / df[0])**(1/n_years) - 1
        ann_vol = returns.std() * np.sqrt(12)
        sharpe = (cagr - risk_free_rate) / ann_vol if ann_vol != 0 else np.nan

        downside_returns = returns.copy()
        downside_returns[downside_returns > 0] = 0
        sortino = (cagr - risk_free_rate) / (downside_returns.std() * np.sqrt(12)) if downside_returns.std() != 0 else np.nan

        running_max = df.cummax()
        drawdown = (df - running_max) / running_max
        max_dd = drawdown.min()
        avg_dd = drawdown.mean()

        # Calmar Ratio = CAGR / Max Drawdown (abs value)
        calmar = cagr / abs(max_dd) if max_dd != 0 else np.nan

        # Drawdown duration
        dd_durations = []
        in_drawdown = False
        dd_start = None

        for date, val in drawdown.items():
            if val < 0:
                if not in_drawdown:
                    in_drawdown = True
                    dd_start = date
            elif in_drawdown:
                in_drawdown = False
                dd_end = date
                dd_durations.append(dd_end - dd_start)

        if in_drawdown:
            dd_durations.append(df.index[-1] - dd_start)

        max_dd_duration = max(dd_durations) if dd_durations else pd.Timedelta(0)
        avg_dd_duration = sum(dd_durations, pd.Timedelta(0)) / len(dd_durations) if dd_durations else pd.Timedelta(0)

        metrics = {
            "Start": start,
            "End": end,
            "Duration": duration,
            "Exposure Time [%]": 100.0,
            "Equity Final [$]": df.iloc[-1] * self.equity,
            "Equity Peak [$]": df.max() * self.equity,
            "Return [%]": cumulative_return * 100,
            "Buy & Hold Return [%]": cumulative_return * 100,
            "Return (Ann.) [%]": cagr * 100,
            "Volatility (Ann.) [%]": ann_vol * 100,
            "CAGR [%]": cagr * 100,
            "Sharpe Ratio": sharpe,
            "Sortino Ratio": sortino,
            "Calmar Ratio": calmar,
            "Alpha [%]": 0.0,  # Requires benchmark comparison
            "Beta": 1.0,       # Requires benchmark comparison
            "Max. Drawdown [%]": max_dd * 100,
            "Avg. Drawdown [%]": avg_dd * 100,
            "Max. Drawdown Duration": max_dd_duration,
            "Avg. Drawdown Duration": avg_dd_duration
        }

        return pd.Series(metrics)

        
    def run(self):

        mtl = self._get_data()
        ret_12, ret_6, ret_3 = self._get_rolling_returns(mtl, self.LOOKBACK_WINDOWS[0], self.LOOKBACK_WINDOWS[1], self.LOOKBACK_WINDOWS[2])
        returns = []
        if len(mtl) < 13:
            raise ValueError("Not enough data to compute momentum strategy.")
            return
        else:
            for i in range(12, len(mtl)-1):
                date = mtl.index[i]
                next_date = mtl.index[i+1]

                top_50 = ret_12.iloc[i-1].nlargest(30).index
                top_30 = ret_6.iloc[i-1][top_50].nlargest(30).index
                top_10 = ret_3.iloc[i-1][top_30].nlargest(10).index

                monthly_ret = mtl.loc[next_date, top_10].mean()
                returns.append(monthly_ret * (1-self.commission))

        strat_pf = pd.Series(returns, index=mtl.index[13:]).cumprod()
        self.strat_pf = strat_pf
        metrics = self._metrics(strat_pf)
        return metrics, pd.Series(returns)


### Mean Reversion

Uses the backtesting.py and pandas-ta libraries

In [ ]:

class mean_reversion_strategy(Strategy):
    # Strategy parameters
    bb_length = 20
    bb_std = 2.0
    rsi_length = 15
    rsi_lower = 30
    rsi_upper = 70
    atr_length = 14
    
    def init(self):        
        price = pd.Series(self.data.Close)
        high = pd.Series(self.data.High)
        low = pd.Series(self.data.Low)

        bb = ta.bbands(close=price, length=self.bb_length, std=self.bb_std)
        rsi = ta.rsi(price, self.rsi_length)
        atr = ta.atr(high, low, price, self.atr_length)
        volume_sma = ta.sma(pd.Series(self.data.Volume), length=20)
        

        self.lower = self.I(lambda x: bb[f'BBL_{self.bb_length}_{self.bb_std}'], 'BB_Lower')
        self.upper = self.I(lambda x: bb[f'BBU_{self.bb_length}_{self.bb_std}'], 'BB_Upper')
        self.middle = self.I(lambda x: bb[f'BBM_{self.bb_length}_{self.bb_std}'], 'BB_Middle')
        self.rsi = self.I(lambda x: rsi, 'RSI')
        self.atr = self.I(lambda x: atr.values, 'ATR')
        self.volume_sma = self.I(lambda x: volume_sma.values, 'Volume Filter')

    def next(self):
        price = self.data.Close[-1]
        volume = self.data.Volume[-1]
        upper = self.upper[-1]
        lower = self.lower[-1]
        rsi = self.rsi[-1]
        atr = self.atr[-1]
        volume_sma = self.volume_sma[-1]
        #print(self.data.index)            
        
        
        if volume < volume_sma:
            return

        if not self.position:
            if price < lower and rsi < self.rsi_lower:
                sl = price - 1.5 * atr
                tp = price + 3.0 * atr
                self.buy(size=0.5)

            elif price> upper and rsi > self.rsi_upper:
                sl = price + 1.5 * atr
                tp = price - 3.0 * atr 
                self.sell(size=0.5, sl=sl, tp=tp)

        #if self.position.is_long


        elif self.position.is_long and price > upper:
            self.position.close()

        elif self.position.is_short and price < lower:
            self.position.close()


## Step 6: Benchmark

Grabs the benchmark data, automatically set to the ^GSPC (S&P) data from yfinance. Gets data from data_loader

In [ ]:

class benchmark:

    def __init__(self, start, end, ticker: str = '^GSPC'):
        self.ticker = ticker
        self.start = datetime.datetime.strptime(start, "%Y-%m-%d")
        self.end = datetime.datetime.strptime(end, "%Y-%m-%d")
        dt = data_loader()
        self.benchmark_data = dt.get_data(self.ticker, self.start, self.end)
        self.daily_returns = self.get_daily_returns()
        self.returns_df = self.benchmark_data[['Close']].copy()
        self.returns_df['Returns'] = self.daily_returns
        self.returns_df.dropna(inplace=True)


    def get_daily_returns(self):
        returns = self.benchmark_data['Close'].pct_change().dropna()
        return returns  # Series

    def get_total_return(self):
        start_price = self.benchmark_data['Close'].iloc[0]
        end_price = self.benchmark_data['Close'].iloc[-1]
        return (end_price / start_price) - 1

    def get_metrics(self, risk_free_rate=0.0):
        df = self.returns_df.copy()
        df['Cumulative'] = (1 + df['Returns']).cumprod()
        df['Peak'] = df['Cumulative'].cummax()
        df['Drawdown'] = df['Cumulative'] / df['Peak'] - 1

        duration = self.end - self.start
        total_return = self.get_total_return()
        annualized_return = (1 + total_return) ** (252 / len(df)) - 1
        volatility = df['Returns'].std() * np.sqrt(252)
        cagr = (df['Cumulative'].iloc[-1]) ** (1 / (len(df) / 252)) - 1
        sharpe = (df['Returns'].mean() * 252 - risk_free_rate) / (df['Returns'].std() * np.sqrt(252))
        downside_returns = df[df['Returns'] < 0]['Returns']
        sortino = (df['Returns'].mean() * 252) / (downside_returns.std() * np.sqrt(252)) if not downside_returns.empty else np.nan
        max_dd = df['Drawdown'].min()
        avg_dd = df['Drawdown'][df['Drawdown'] < 0].mean()

        # Drawdown duration
        drawdown_durations = []
        current_dd_duration = 0
        for dd in df['Drawdown']:
            if dd < 0:
                current_dd_duration += 1
            else:
                if current_dd_duration > 0:
                    drawdown_durations.append(current_dd_duration)
                    current_dd_duration = 0
        if current_dd_duration > 0:
            drawdown_durations.append(current_dd_duration)
        max_dd_duration = max(drawdown_durations) if drawdown_durations else 0
        avg_dd_duration = np.mean(drawdown_durations) if drawdown_durations else 0

        # Alpha & Beta vs market (self vs itself here, but you could modify this to compare with another benchmark)
        market_returns = df['Returns']
        slope, intercept, r_value, p_value, std_err = linregress(market_returns, df['Returns'])
        beta = slope
        alpha = (annualized_return - risk_free_rate) - beta * (annualized_return - risk_free_rate)

        return pd.Series({
            'Start': self.start,
            'End': self.end,
            'Duration': duration,
            'Exposure Time [%]': 100.0,  # Always exposed
            'Equity Final [$]': df['Cumulative'].iloc[-1] * 10000,
            'Equity Peak [$]': df['Peak'].max() * 10000,
            'Return [%]': total_return * 100,
            'Buy & Hold Return [%]': total_return * 100,
            'Return (Ann.) [%]': annualized_return * 100,
            'Volatility (Ann.) [%]': volatility * 100,
            'CAGR [%]': cagr * 100,
            'Sharpe Ratio': sharpe,
            'Sortino Ratio': sortino,
            'Calmar Ratio': cagr / abs(max_dd) if max_dd != 0 else np.nan,
            'Alpha [%]': alpha * 100,
            'Beta': beta,
            'Max. Drawdown [%]': max_dd * 100,
            'Avg. Drawdown [%]': avg_dd * 100,
            'Max. Drawdown Duration': pd.Timedelta(days=int(max_dd_duration)),
            'Avg. Drawdown Duration': pd.Timedelta(days=int(avg_dd_duration)),
        })


## Step 7: Data Loader

Fetches the data used to power this system from  yfinance or the local disc, once data is fetched once it stores it in the data/raw folder to ensure dublicate data is note gotten. This speeds up the systema and removes the impact on the api.

In [ ]:
class data_loader:

    def __init__(self):
        self.raw_path = "./data/raw"
        self.processed_path = "./data/processed"

    def _raw_filepath(self, ticker: str) -> str:
        return os.path.join(self.raw_path, f"{ticker}.csv")

    def get_data(self, ticker: str, start: str, end: str) -> pd.DataFrame:
        ticker = ticker.replace('.', '-')

        filepath = self._raw_filepath(ticker)

        if os.path.exists(filepath):
            data = pd.read_csv(filepath, index_col='Date', parse_dates=True)
            sliced_data = data.loc[start:end]
            print(f"Retrieving {ticker} data from csv")
            if not sliced_data.empty:
                return sliced_data

        print(f"Downloading {ticker} from yfinance...")
        data = yf.download(ticker, start=start, end=end, progress=False)
        if data.empty:
            print(f"[!] Failed to download {ticker}. Skipping.")
            return None
        data.dropna(inplace=True)
        if 'Adj Close' in data.columns:
            data.drop(columns=['Adj Close'], inplace=True)
        #data.rename(columns={'Open': 'open','High': 'high','Low': 'low','Close': 'close','Volume': 'volume'}, inplace=True)
        #required_cols = ['open', 'high', 'low', 'close', 'volume']
        #data = data[required_cols]
        data = data.droplevel('Ticker', axis=1)
        data.reset_index(inplace=True)
        data.index = data['Date']
        del data['Date']
        data.index.name = 'Date'
        data.to_csv(filepath)
        return data.loc[start:end]


    def get_multiple_data(self, tickers: List[str], start: str, end: str) -> dict:
        data_dict = {}
        for ticker in tickers:
            df = self.get_data(ticker, start, end)
            if df is not None:
                data_dict[ticker] = df

        return data_dict
    
    def get_sp500_data(self, start: str, end: str):
        sp500 = pd.read_html("https://en.wikipedia.org/wiki/List_of_S%26P_500_companies")[0]
        sp500 = sp500['Symbol'].to_list()
        data = self.get_multiple_data(sp500, start, end)
        return data

    def get_sp500_data_df(self, start: str, end: str) -> pd.DataFrame:
        tickers = pd.read_html("https://en.wikipedia.org/wiki/List_of_S%26P_500_companies")[0]['Symbol'].tolist()
        tickers = [t.replace('.', '-') for t in tickers]
        data_dict = self.get_multiple_data(tickers, start, end)
        close_prices = [df[['Close']].rename(columns={'Close': ticker}) for ticker, df in data_dict.items()]
        return pd.concat(close_prices, axis=1) if close_prices else pd.DataFrame()
    
    def get_fundamentals(self, ticker):
        ticker = ticker.replace('.', '-')
        file_path = os.path.join(self.raw_path, f"{ticker}_fundamentals.json")

        if os.path.exists(file_path):
            try:
                with open(file_path, 'r') as f:
                    fundamentals = json.load(f)
                print(f"[!] Loading fundamentals from file for {ticker}")
                return fundamentals
            except Exception as e:
                print(f"[!] Failed to load fundamentals from file for {ticker}: {e}")

        try:
            print(f"[i] Downloading fundamentals for {ticker} from Yahoo Finance... ")
            fundamentals = yf.Ticker(ticker).info
            with open(file_path, 'w') as f:
                json.dump(fundamentals, f)
            return fundamentals
        except Exception as e:
            print(f"[!] Failed to download fundamentals for {ticker}: {e}")
            return {}


## Step 8: Backtester

Used in the portfolio specifically for the mean reversion strategy. This class uses the backtesting class from backtesting.py and just adds a few more methods to extend it

In [ ]:
class GenericBacktestEngine:
    def __init__(self, strategy_cls, strategy_kwargs: dict = None, cash: float = 10000, commission: float = 0.002):
        """
        Initializes the backtest engine.

        :param data: Historical OHLCV DataFrame.
        :param strategy_cls: A class inheriting from backtesting.Strategy.
        :param strategy_kwargs: Optional kwargs to be passed to the strategy.
        :param cash: Starting cash for the backtest.
        :param commission: Commission per trade (e.g., 0.002 = 0.2%).
        """
        self.strategy_cls = strategy_cls
        self.strategy_kwargs = strategy_kwargs
        self.cash = cash
        self.commission = commission

    def run(self, data: pd.DataFrame):
        """
        Runs the backtest and returns results.
        """
        
        bt = Backtest(
            data,
            self.strategy_cls,
            cash=self.cash,
            commission=self.commission
        )
        stats = bt.run()
        equity_curve = stats['_equity_curve']
        daily_returns = equity_curve['Equity'].pct_change().dropna()
        return stats, daily_returns

    def plot(self, data: pd.DataFrame):
        """
        Plots the results using backtesting.py's built-in plot function.
        Saves the output as an HTML file and returns the relative path.
        """

        strategy = self.strategy_cls.__name__
        start_date = str(data.index[0])[:10]
        end_date = str(data.index[-1])[:10]

        plot_filename = f"{strategy}_{start_date}--{end_date}.html"
        plot_path = os.path.join("reporting", "charts", plot_filename)

        bt = Backtest(
            data,
            self.strategy_cls,
            cash=self.cash,
            commission=self.commission
        )
        bt.run(**(self.strategy_kwargs or {}))

        bt.plot(filename=plot_path, open_browser=False)

        return plot_path

    def batch_backtest(self, data_dict: dict):
        """
        Run backtests on a dict of ticker: DataFrame pairs.
        Returns a dict of ticker: stats
        """
        results = {}
        pf_returns = {}
        for ticker, data in data_dict.items():
            try:
                stats, pf_ret = self.run(data)
                pf_returns[ticker] = pf_ret
                results[ticker] = stats
            except Exception as e:
                print(f"Failed on {ticker}: {e}")
        return results, pf_returns


## Step 9: Dashboard

Finally this is the web dashboard used for the client side. Currently in MVP form and functions with very basic customization and return values, nonetheless works find

In [ ]:

from portfolio.construction import Portfolio

# -- Streamlit UI Setup --

st.set_page_config(page_title='ASG Microfund Dashboard', layout='wide')
st.title("ASG Microfund Preformance Dashboard")

# -- Sidebar: User Inputs --
st.sidebar.header("Simulation Configuration")

start_date = st.sidebar.text_input("Start Date", '2020-01-01')
end_date = st.sidebar.text_input("End Date", '2025-01-01')

risk_tol = st.sidebar.text_input("Risk Tolerance","medium")
time_hor = st.sidebar.text_input("Time Horizon","medium")

commissions = st.sidebar.slider("Commissions (%)", 0.0, 10.0, 0.1, 0.01)
cash = st.sidebar.number_input("Starting Cash ($)", min_value=1000, max_value=1_000_000, value=100_000, step=1000)

mean_tickers = st.sidebar.text_area("Mean Reversion Tickers (comma-separated)", "AAPL,MSFT,GOOGL")
factor_tickers = st.sidebar.text_area("Factor Investing Tickers (comma-separated)", "BRK-B,JNJ,UNH")
mean_ticker_list = [t.strip().upper() for t in mean_tickers.split(",") if t.strip()]
factor_ticker_list = [t.strip().upper() for t in factor_tickers.split(",") if t.strip()]

run_simulation = st.sidebar.button("Run Portfolio Backtest")

if run_simulation:
    with st.spinner("Running portfolio backtest..."):
        try:
            port = Portfolio(
                start=start_date,
                end=end_date,
                commissions=(commissions/100),
                cash=cash,
                mean_tickers=mean_ticker_list,
                factor_tickers=factor_ticker_list,
                user_tolerance=risk_tol,
                user_time=time_hor
            )

            mean_results, mom_results, factor_results, final_metrics, benchmark_metrics, returns_df = port.backtest_portfolio()



            benchmark_metrics_sum = {}
            keys = ['Return [%]', 'CAGR [%]', 'Max. Drawdown [%]']
            for key in keys:
                benchmark_metrics_sum[key] = benchmark_metrics[key]

            st.success("Backtest complete!")

            # Display Metrics

            st.header("Final Portfolio Metrics")
            st.dataframe(pd.DataFrame(final_metrics, index=["Metrics"]))

            st.header("Benchmark Comparison")
            st.dataframe(benchmark_metrics_sum)

            st.header("Strategy Summaries")
            st.subheader("Mean Reversion")
            st.dataframe(pd.DataFrame(mean_results).T)

            st.subheader("Momentum")
            st.dataframe(mom_results)

            st.subheader("Factor Investing")
            st.dataframe(factor_results)

            st.subheader("Overall Daily Strategy Preformance")
            st.dataframe(returns_df)

            #st.download_button("Download Report", )

        except Exception as e:
            st.error(f"Error during simulation: {e}")

        
            
else:
    st.info("Configure inputs in the sidebar and click 'Run Portfolio Backtest' to begin.")


